In [ ]:
import os
from PIL import Image
import pytesseract
import re
from datetime import datetime
import mysql.connector
import time

# Path to Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# MySQL configuration
mysql_config = {
    'host': "host name",
    'user': "user name",
    'password': "password",
    'database': "db name"
}

# Connect to MySQL
def connect_to_database():
    try:
        conn = mysql.connector.connect(**mysql_config)
        cursor = conn.cursor()
        print("Connected to MySQL!")
        return conn, cursor
    except mysql.connector.Error as e:
        print(f"Error connecting to MySQL: {e}")
        exit()

conn, cursor = connect_to_database()

# Folder containing images
folder_path = r"folder path"

# Coordinates for the region to crop (left, upper, right, lower)
specified_region_coordinates = (0, 0, 500, 500)

# Set to track processed files
processed_files = set()

# Function to extract value from the image
def extract_value_from_image(image):
    # Use pytesseract to extract text from the specified region in the image
    cropped_img = image.crop(specified_region_coordinates)
    text = pytesseract.image_to_string(cropped_img)
    matches = re.findall(r'([-+]?\d{1,3}(?:,\d{3})*\.\d{2})', text)
    if matches:
        return matches[0]  # Return the first match (including sign)
    return None

while True:
    files_in_folder = os.listdir(folder_path)
    new_files = [filename for filename in files_in_folder if filename not in processed_files]

    for filename in new_files:
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.avif')):
            img = Image.open(file_path)
            modified_time = datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y-%m-%d %H:%M:%S')
            pnl_value = extract_value_from_image(img)
            insert_query = "INSERT INTO 5paisa (filename, modified_time, pnl_value) VALUES (%s, %s, %s)"
            data = (filename, modified_time, pnl_value)
            try:
                cursor.execute(insert_query, data)
                conn.commit()
                print(f"Data inserted for File: {filename}")
            except mysql.connector.Error as e:
                print(f"Error inserting data for File {filename}: {e}")
            processed_files.add(filename)

    time.sleep(10)

cursor.close()
conn.close()
